In [3]:
import xarray as xr, matplotlib.pyplot as plt, cartopy, pandas as pd, numpy as np, regionmask, re
xr.set_options(keep_attrs = True)

from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta

from IPython.display import clear_output


ERROR 1: PROJ: proj_create_from_database: Open of /rds/general/user/cb2714/home/miniforge3/envs/wwa/share/proj failed


# NAO

## Pre-processing

### ERA5 from MSLP

In [ ]:
# MSLP fields to calculate manually
! wget https://climexp.knmi.nl/dyn_links/era5_msl_-28--16E_36-70N_-999-lastyear_su.nc -O data/era5_msl_-28--16E_36-70N.nc

clear_output(wait = False)

In [3]:
mslp = wrap_lon(decode_times(xr.open_dataset("data/era5_msl_-28--16E_36-70N.nc", decode_times = False)))

In [4]:
slp_n = awmean(mslp.sel(lon = slice(-25,-16), lat = slice(63,70)).msl)
slp_s = awmean(mslp.sel(lon = slice(-28,-20), lat = slice(36,40)).msl)

nao_raw = slp_s - slp_n

nao_z = (nao_raw - nao_raw.mean()) / nao_raw.std()

nao_z.to_dataframe().to_csv("data/nao-stn_monthly_era5.csv")

### Pre-computed indices

In [ ]:
! wget https://www.cpc.ncep.noaa.gov/products/precip/CWlink/pna/norm.nao.monthly.b5001.current.ascii -O data/nao-pc_monthly_noaa.txt
! wget https://downloads.psl.noaa.gov/Public/map/teleconnections/nao.reanalysis.t10trunc.1948-present.txt -O data/nao-pc_daily_noaa.txt
! wget https://climatedataguide.ucar.edu/sites/default/files/2023-07/nao_station_monthly.txt -O data/nao-stn_monthly_ucar.txt

clear_output(wait = False)

In [70]:
# monthly from PC - NOAA
nao = pd.read_csv("data/nao-pc_monthly_noaa.txt", sep = " +", header = None, names = ["year", "month", "nao"], engine = "python")
nao["day"] = 1
nao.index = pd.to_datetime(nao[["year", "month", "day"]])

nao[["nao"]].to_csv("data/nao-pc_monthly_noaa.csv")

In [ ]:
# daily from PC - NOAA
nao = pd.read_csv("data/nao-pc_daily_noaa.txt", sep = " +", header = None, names = ["year", "month", "day", "nao"], engine = "python")
nao.index = pd.to_datetime(nao[["year", "month", "day"]])

In [186]:
# monthly from stations - UCAR
nao = pd.read_csv("data/nao-stn_monthly_ucar.txt", sep = " +", engine = "python", skiprows = 1)
nao.columns = range(1,13)
nao[nao == -999] = np.nan
nao = nao.unstack().reset_index().rename(columns = {"level_1" : "year", "level_0" : "month", 0 : "nao"})
nao["day"] = 1
nao.index = pd.to_datetime(nao[["year", "month", "day"]])
nao = nao.dropna(axis = 0, how = "any")
nao = nao.sort_index()[["nao"]]

nao.to_csv("data/nao-stn_monthly_ucar.csv")

## DJF time series for ClimExp

### NOAA (PC)

In [73]:
# csv for climate explorer
csv_fnm = "med-storms_nao-djf_noaa-pc"
csv_fullpath = "ts/"+csv_fnm+".dat"
if len(csv_fnm) > 61: print("! Filename too long: ", csv_fnm)

# create header lines for upload to Climate Explorer 
str1 = "# contact :: December-February mean of NAO index - Mediterranean Storms, c.barnes22@imperial.ac.uk"
str2 = "# nao [] DJF NAO from https://www.cpc.ncep.noaa.gov/products/precip/CWlink/pna/norm.nao.monthly.b5001.current.ascii at "+csv_fnm+".dat"
head = "# year nao"

# make .dat file
! echo "$str1 " > $csv_fullpath
! echo "$str2" >> $csv_fullpath
! echo "$head" >> $csv_fullpath
nao_djf[["nao"]].to_csv(csv_fullpath, sep = " ", mode = "a", header = False)

### ERA5

In [10]:
nao = pd.read_csv("data/nao-stn_monthly_era5.csv", index_col = 0, parse_dates = [0])
nao_djf = nao.loc[nao.index.month.isin([1,2,12])].resample("YE-FEB").mean()
nao_djf.index = nao_djf.index.year

In [ ]:
# csv for climate explorer
csv_fnm = "med-storms_nao-djf_era5-stn"
csv_fullpath = "ts/"+csv_fnm+".dat"
if len(csv_fnm) > 61: print("! Filename too long: ", csv_fnm)

# create header lines for upload to Climate Explorer 
str1 = "# contact :: December-February mean of NAO index from ERA5 MSLP - Mediterranean Storms, c.barnes22@imperial.ac.uk"
str2 = "# nao [] DJF NAO: normalised difference between area-weighted mean MSLP in [28W-20W, 36N-40N] and [25W-16W, 63N-70N] at "+csv_fnm+".dat"
head = "# year nao"

# make .dat file
! echo "$str1 " > $csv_fullpath
! echo "$str2" >> $csv_fullpath
! echo "$head" >> $csv_fullpath
nao_djf.to_csv(csv_fullpath, sep = " ", mode = "a", header = False)

# Extract MSWEP data

In [4]:
# extract MSWEP for this year only
xn,xx,yn,yx = [-10,20,32,50]
pr = xr.open_mfdataset("/rds/general/user/cb2714/home/00_WWA_project_folder/live/data/mswep/2026/*.nc").precipitation.sel(lon = slice(xn,xx), lat = slice(yx,yn))
rm = regionmask.defined_regions.natural_earth_v5_1_2.land_50.mask(pr)
pr = pr.where(rm == 0)
pr.to_netcdf("data/mswep_2026.nc")

# Convert Mariam time series to .csv

In [ ]:
ts1 = xr.open_dataset("/rds/general/user/cb2714/home/00_WWA_project_folder/ephemeral/mediterranean-storms/obs/ERA5_Kris_daily-ts.nc").tp.rename("pr_n")
ts2 = xr.open_dataset("/rds/general/user/cb2714/home/00_WWA_project_folder/ephemeral/mediterranean-storms/obs/ERA5_LeMa_daily-ts.nc").tp.rename("pr_s")

df = xr.merge([ts1, ts2]).to_dataframe()
df.to_csv("data/ts_pr_era5x.csv")

In [31]:
rx1day = df.resample("YE-JUL").max()
rx1day.index = rx1day.index.year
rx1day.index.name = "year"

rx1day_djf = df.loc[df.index.month.isin([12,1,2])].resample("YE-JUL").max()
rx1day_djf.index = rx1day_djf.index.year
rx1day_djf.index.name = "year"

rx3day = df.rolling(3).sum().resample("YE-JUL").max()
rx3day.index = rx3day.index.year
rx3day.index.name = "year"

rx3day_djf = df.rolling(3).sum().loc[df.index.month.isin([12,1,2])].resample("YE-JUL").max()
rx3day_djf.index = rx3day_djf.index.year
rx3day_djf.index.name = "year"

In [ ]:
rx1day.to_csv("data/rx1day_era5.csv")
rx1day_djf.to_csv("data/rx1day-djf_era5.csv")
rx3day.to_csv("data/rx3day_era5.csv")
rx3day_djf.to_csv("data/rx3day-djf_era5.csv")

In [ ]:
ts = xr.open_dataset("/rds/general/user/cb2714/home/00_WWA_project_folder/ephemeral/mediterranean-storms/obs/ERA5_Box3_daily-ts.nc")
ts.to_dataframe().to_csv("data/ts_pr_era5x_box3.csv")